In [1]:
#Importing Libraries 
import pyspark
from pyspark.sql import SparkSession
import argparse
import os 
import re
import pandas as pd 


In [ ]:

# Starting a sparksession 
spark = SparkSession.builder\
                    .master("local")\
                    .appName('Firstprogram')\
                    .getOrCreate()
sc=spark.sparkContext

In [48]:
# Read the input file and Calculating words count
load_file = "biographies.txt"
writeFile = open('updated_.txt', 'w')
# Filterting the lines which starts with 'BG:'
with open('biographies.txt', encoding = "ISO-8859-1") as f:
  for line in f:
    try:
      if not (line.startswith('BG:')): 
        line = re.sub('[^A-Za-z0-9]+', ' ', line)
        line = re.sub(r'[0-9]', ' ', line)
        print(line) 
        # break

        writeFile.write(line) # Saving the filtered lines in a seperate file as updateFile.txt
    except UnicodeDecodeError:
      pass

In [49]:
#Loading the data file
text_file = sc.textFile('updated_.txt')
counts = text_file.flatMap(lambda line: line.split(" ")) \
                            .map(lambda word: (word, 1)) \
                           .reduceByKey(lambda x, y: x + y)

In [50]:
output = counts.collect()

data_df = list()
for (word, count) in output:
  data_df.append((word, count))


In [51]:
import pandas as pd 
df = pd.DataFrame(data_df, columns=['word', 'count'])

print(df)

             word    count
0                  5127742
1             NM:   664805
2               *  2005591
3             (17     2219
4        official     1297
...           ...      ...
1727152     Tozlu        1
1727153    Aybüke        1
1727154    Üstel,        1
1727155      Poj,        1
1727156   Bankasi        1

[1727157 rows x 2 columns]


In [52]:
df.to_csv("word_count.csv")

In [40]:
# Stopping Spark-Session and Spark context
sc.stop()
spark.stop()